In [49]:
# Install MySql packages
# conda update -n base -c conda-forge conda --yes
# !pip install ipython-sql
# !pip install mysql-connector-python
# !conda install -c conda-forge python-dotenv --yes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import mysql.connector
from mysql.connector import Error
from os import environ, path
from dotenv import load_dotenv

In [8]:
# Read enviromental variables to connect the database
load_dotenv()
password = environ.get('PASSWORD')
host = environ.get('HOST')
database = environ.get('DATABASE')

## Connect to the database and create dataframes for each table

In [9]:
try:
    connection = mysql.connector.connect(host=host ,
                                        database=database,
                                        user='root',
                                        password=password)
    if connection.is_connected():
        print("Succes Connected")
    df_musician = pd.read_sql_query("SELECT * FROM music_and_death.musician;", connection)
    df_birth = pd.read_sql_query("SELECT * FROM music_and_death.birth;", connection)
    df_death = pd.read_sql_query("SELECT * FROM music_and_death.death;", connection)
    df_genre = pd.read_sql_query("SELECT * FROM music_and_death.music_genre;", connection)
    genre = pd.read_sql_query("SELECT * FROM music_and_death.genre;", connection)
    cause_death = pd.read_sql_query("SELECT * FROM music_and_death.cause_death;", connection)
    cities = pd.read_sql_query("SELECT * FROM music_and_death.cities;", connection)
    states = pd.read_sql_query("SELECT * FROM music_and_death.states;", connection)
    countries = pd.read_sql_query("SELECT * FROM music_and_death.countries;", connection)
    
    
except Error as e:
    print("Error closing connection")

# Concatenate tables
else:
    # Concatenate musician and death dataframes
    df = pd.concat([df_musician, df_birth], axis=1)
    df = pd.concat([df, df_death], axis=1)
    df = pd.merge(df, cause_death, on=["death_id", "death_id"])
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df = pd.merge(df, df_genre, on=["musician_id"])
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df = pd.merge(df, genre, on=["genre_id"])
    # rearrange columns
    df = df.iloc[:, [0, 1, 2, 3, 5, 7, 8, 9, 11, 6, 4, 10]]
    
finally:
    connection.close()

Succes Connected


In [10]:
df

,musician_id,stage_name,middle_name,stage_lastname,birth,death_date,death_age,cause,genre,birthplace_id,death_id,genre_id
0,1,John,Winston,Lennon,1940-10-09,1980-12-08,40,homicide,rock,50337,1,2
1,2,Kurt,Donald,Cobain,1967-02-20,1994-04-05,27,suicide,rock,110979,2,2
2,17,Chris,John,Cornell,1964-07-20,2017-05-18,52,suicide,rock,126104,2,2
3,18,Chester,Charles,Bennington,1976-03-20,2017-07-20,41,suicide,rock,124148,2,2
4,32,Ian,Kevin,Curtis,1956-07-15,1980-05-18,23,suicide,rock,51499,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
128,119,Sol,None,Hoʻopiʻi,1902-12-19,1953-11-16,51,diabetes,blues,118623,13,11
129,112,Papa,Charlie,McCoy,1909-05-26,1950-07-26,41,brain disorder,blues,119065,15,11
130,115,Luke,,Jordan,1892-01-28,1952-06-25,58,not reported,blues,120916,16,11
131,127,Ralph,,Willis,1910-01-01,1957-06-11,47,not reported,blues,112358,16,11


## Add birth places

In [11]:
pd.concat([df, df_birth], axis=1)

,musician_id,stage_name,middle_name,stage_lastname,birth,death_date,death_age,cause,genre,birthplace_id,death_id,genre_id,musician_id,birth,birthplace_id
0,1,John,Winston,Lennon,1940-10-09,1980-12-08,40,homicide,rock,50337,1,2,1,1940-10-09,50337
1,2,Kurt,Donald,Cobain,1967-02-20,1994-04-05,27,suicide,rock,110979,2,2,2,1967-02-20,110979
2,17,Chris,John,Cornell,1964-07-20,2017-05-18,52,suicide,rock,126104,2,2,3,1946-11-16,109304
3,18,Chester,Charles,Bennington,1976-03-20,2017-07-20,41,suicide,rock,124148,2,2,4,1966-08-20,116141
4,32,Ian,Kevin,Curtis,1956-07-15,1980-05-18,23,suicide,rock,51499,2,2,5,1956-12-06,125919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,119,Sol,None,Hoʻopiʻi,1902-12-19,1953-11-16,51,diabetes,blues,118623,13,11,129,1902-04-09,122067
129,112,Papa,Charlie,McCoy,1909-05-26,1950-07-26,41,brain disorder,blues,119065,15,11,130,1873-11-16,116577
130,115,Luke,,Jordan,1892-01-28,1952-06-25,58,not reported,blues,120916,16,11,131,1923-02-01,124477
131,127,Ralph,,Willis,1910-01-01,1957-06-11,47,not reported,blues,112358,16,11,132,1903-06-26,119176


In [12]:
df_2 = df.drop(['middle_name', 'death_id', "death_date", "genre_id"], axis=1)

In [13]:
df_2

,musician_id,stage_name,stage_lastname,birth,death_age,cause,genre,birthplace_id
0,1,John,Lennon,1940-10-09,40,homicide,rock,50337
1,2,Kurt,Cobain,1967-02-20,27,suicide,rock,110979
2,17,Chris,Cornell,1964-07-20,52,suicide,rock,126104
3,18,Chester,Bennington,1976-03-20,41,suicide,rock,124148
4,32,Ian,Curtis,1956-07-15,23,suicide,rock,51499
...,...,...,...,...,...,...,...,...
128,119,Sol,Hoʻopiʻi,1902-12-19,51,diabetes,blues,118623
129,112,Papa,McCoy,1909-05-26,41,brain disorder,blues,119065
130,115,Luke,Jordan,1892-01-28,58,not reported,blues,120916
131,127,Ralph,Willis,1910-01-01,47,not reported,blues,112358


In [20]:
cities.shape

(148266, 3)

In [21]:
states.shape

(4979, 3)

In [22]:
countries.shape

(250, 4)

In [23]:
cities

,id,name,state_id
0,52,Ashkāsham,3901
1,68,Fayzabad,3901
2,78,Jurm,3901
3,84,Khandūd,3901
4,115,Rāghistān,3901
...,...,...,...
148261,131496,Redcliff,1957
148262,131502,Shangani,1957
148263,131503,Shurugwi,1957
148264,131504,Shurugwi District,1957


In [24]:
states

,id,name,country_id
0,3901,Badakhshan,1
1,3871,Badghis,1
2,3875,Baghlan,1
3,3884,Balkh,1
4,3872,Bamyan,1
...,...,...,...
4974,1953,Mashonaland West Province,247
4975,1960,Masvingo Province,247
4976,1954,Matabeleland North Province,247
4977,1952,Matabeleland South Province,247


In [25]:
countries

,id,name,iso2,capital
0,1,Afghanistan,AF,Kabul
1,2,Aland Islands,AX,Mariehamn
2,3,Albania,AL,Tirana
3,4,Algeria,DZ,Algiers
4,5,American Samoa,AS,Pago Pago
...,...,...,...,...
245,243,Wallis And Futuna Islands,WF,Mata Utu
246,244,Western Sahara,EH,El-Aaiun
247,245,Yemen,YE,Sanaa
248,246,Zambia,ZM,Lusaka


In [27]:
cities.columns

Index(['id', 'name', 'state_id'], dtype='object')

In [32]:
result = cities.merge(states, left_on=cities["state_id"], right_on=states["id"])

In [44]:
result[117000:173500]

,key_0,id_x,name_x,state_id,id_y,name_y,country_id
117000,1662,17485,Bezirk Gäu,1662,1662,Solothurn,214
117001,1662,17486,Bezirk Gösgen,1662,1662,Solothurn,214
117002,1662,17495,Bezirk Lebern,1662,1662,Solothurn,214
117003,1662,17503,Bezirk Olten,1662,1662,Solothurn,214
117004,1662,17511,Bezirk Solothurn,1662,1662,Solothurn,214
...,...,...,...,...,...,...,...
148261,1957,131496,Redcliff,1957,1957,Midlands Province,247
148262,1957,131502,Shangani,1957,1957,Midlands Province,247
148263,1957,131503,Shurugwi,1957,1957,Midlands Province,247
148264,1957,131504,Shurugwi District,1957,1957,Midlands Province,247
